In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path

from sdm_eurec4a.visulization import set_custom_rcParams, label_from_attrs
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

default_colors = set_custom_rcParams()

import matplotlib.pyplot as plt

In [ ]:
data_dir = Path(
    "/home/m/m301096/CLEO/data/output_v4.0/coalbure_condensation_large/combined/conservation_dataset_combined.nc"
)
ds = xr.open_dataset(data_dir)
ds = ds.isel(time=slice(0, -1))

ds["sum"] = -ds["reservoir_change"] + ds["inflow"] + ds["outflow"] + ds["source"]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=False)

axs = axs.flatten()

for cloud_id in ds["cloud_id"]:

    axs[0].plot(
        ds["time"],
        ds["inflow"].sel(cloud_id=cloud_id),
        label=label_from_attrs(ds["cloud_id"].sel(cloud_id=cloud_id)),
    )
    axs[1].plot(
        ds["time"],
        ds["outflow"].sel(cloud_id=cloud_id),
        label=label_from_attrs(ds["cloud_id"].sel(cloud_id=cloud_id)),
    )
    axs[2].plot(
        ds["time"],
        ds["source"].sel(cloud_id=cloud_id),
        label=label_from_attrs(ds["cloud_id"].sel(cloud_id=cloud_id)),
    )
    axs[3].plot(
        ds["time"],
        ds["sum"].sel(cloud_id=cloud_id),
        label=label_from_attrs(ds["cloud_id"].sel(cloud_id=cloud_id)),
    )

In [ ]:
microphysics = (
    # 'null_microphysics',
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)
for mp, color in zip(microphysics, default_colors):

    data_dir = Path(
        f"/home/m/m301096/CLEO/data/output_v4.0/{mp}/combined/conservation_dataset_combined.nc"
    )
    ds = xr.open_dataset(data_dir)
    ds = ds.isel(time=slice(0, -1))

    ds["sum"] = -ds["reservoir_change"] + ds["inflow"] + ds["outflow"] + ds["source"]

    ds_mean, ds_std = mean_and_stderror_of_mean(
        ds.sel(time=slice(1000, 3400)),
        dims="time",
    )

    # ds_mean = ds.sel(time = slice(1000, 3400)).mean('time')
    # ds_std = ds.sel(time = slice(1000, 3400)).std('time')

    A = -100 * ds_mean["source"]
    B = ds_mean["inflow"]
    sigma_A = -100 * ds_std["source"]
    sigma_B = ds_std["inflow"]

    f = A / B
    sigma_f = f * np.sqrt((sigma_A / A) ** 2 + (sigma_B / B) ** 2)

    m, s = mean_and_stderror_of_mean(data=f, dims="cloud_id", data_std=sigma_f)

    plt.hist(f, alpha=0.1, bins=np.linspace(-5, 20, 50), color=color)
    plt.axvline(m, label=mp, alpha=1, color=color)
    plt.fill_betweenx([0, 20], m - s, m + s, alpha=0.2, color=color)
    # plt.hist(out_inf, label = mp, alpha = 0.5, bins = np.linspace(-100, 100, 50))
    # plt.plot(source - out_inf, label = mp, alpha = 0.5)

plt.legend()

In [ ]:
ds

In [ ]:
microphysics = (
    # 'null_microphysics',
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

for mp, color, marker in zip(microphysics, default_colors, [".", "x", "+", "1", "2"]):

    data_dir = Path(
        f"/home/m/m301096/CLEO/data/output_v4.0/{mp}/combined/conservation_dataset_combined.nc"
    )
    ds = xr.open_dataset(data_dir)
    ds = 0.5 * ds.isel(time=slice(0, -1))

    ds["sum"] = -ds["reservoir_change"] + ds["inflow"] + ds["outflow"] + ds["source"]

    ds_mean, ds_std = mean_and_stderror_of_mean(
        ds.sel(time=slice(1000, 3400)),
        dims="time",
    )
    # ds_mean = ds.sel(time = slice(1000, 3400)).mean('time')
    # ds_std = ds.sel(time = slice(1000, 3400)).std('time')

    # ds_mean = ds.sel(time = slice(1000, 3400)).mean('time')
    # ds_std = ds.sel(time = slice(1000, 3400)).std('time')

    A = -100 * ds_mean["source"]
    B = ds_mean["inflow"]
    sigma_A = -100 * ds_std["source"]
    sigma_B = ds_std["inflow"]

    f = A / B
    sigma_f = f * np.sqrt((sigma_A / A) ** 2 + (sigma_B / B) ** 2)
    # sigma_f = ds_std['source']
    # f = -ds_mean['source']

    m, s = mean_and_stderror_of_mean(data=f, dims="cloud_id", data_std=sigma_f)

    plt.errorbar(
        x=ds_mean["inflow"],
        y=f,
        yerr=sigma_f,
        color=color,
        alpha=0.8,
        marker=marker,
        linestyle="None",
    )
    plt.axhline(m, label=mp, alpha=1, color=color, linestyle="--")
    # plt.fill_between([0, 100], m - 2* s, m + 2*s, alpha = 0.2, color = color)
    # plt.hist(out_inf, label = mp, alpha = 0.5, bins = np.linspace(-100, 100, 50))
    # plt.plot(source - out_inf, label = mp, alpha = 0.5)

plt.legend(loc="lower left")
plt.xscale("log")
plt.yscale("log")